In [1]:
import pandas as pd
import numpy as np
import csv
import pickle
from collections import Counter
import matplotlib.pyplot as plt
import powerlaw,math
import matplotlib

In [2]:
import seaborn as sns

In [3]:
closure_opening = pd.read_csv('./Closure and re-opening NPI among countries.csv') 

In [4]:
closure_opening = np.array(closure_opening)

In [5]:
countries = closure_opening[:,1]
region = closure_opening[:,4]
time_intervention = closure_opening[:,5]
social_return = closure_opening[:,8]
set_countries = list(set(countries))
set_countries = [k for k in set_countries if type(k)!=float]

In [6]:
#By row
country_region_time_intervention = {}
for k in set_countries:
    time_tp = time_intervention[countries==k]        #length
    region_tp = region[countries==k]                 #region and country
    social_return_tp = social_return[countries==k]   #intervention
    region_tp_set = list(set(region_tp))             #regions in the same country
    #By regions
    region_set = {}
    if len(region_tp_set)>1:  #multipul regions
        for k1 in region_tp_set:
            time_tp1 = time_tp[region_tp==k1]                     #intervention in a region
    #         print(timp_tp1)
            social_return_tp1 = social_return_tp[region_tp==k1]   #intervention in a region
            region_set[k1] = [(time_tp1[i],social_return_tp1[i]) for i in range(len(time_tp1))]
    else:
        tp_i = []
        for i in range(len(social_return_tp)):
            tp_i.append((time_tp[i],social_return_tp[i]))
        region_set[region_tp_set[0]] = tp_i 
    country_region_time_intervention[k] = region_set

In [7]:
chs = ['Re-opening of kindergartens','Re-opening of primary schools','Closure of primary and secondary schools',
'Re-opening of primary schools and child care','Complete closure of all educational institutions',
'Closure of kindergartens','Re-opening of child care centers',
 'Re-opening of secondary schools','Re-opening of primary and secondary schools']

In [8]:
#new 
certain_info = {}
for k in country_region_time_intervention:
    tp = country_region_time_intervention[k]
    tp_dict = {}
    for k1 in tp:
        tp1 = []
        for k2 in tp[k1]:   #situation in a region
            if k2[1] in chs:
                tp1.append(k2)
        tp_dict[k1] = tp1
    certain_info[k] = tp_dict

In [9]:
country_region_time_intervention_sorted = {}
for k in certain_info:
    tp = certain_info[k]
    tp_dict = {}
    for k1 in tp:
        tp1 = tp[k1]
        tp1 = sorted(tp1, key=lambda item: int(item[0].split('/')[0])*30+int(item[0].split('/')[1]))
        tp_dict[k1] = tp1
   
    country_region_time_intervention_sorted[k] = tp_dict

In [10]:
country_region_01 = {}    #thansfering into 1 and 0
for k in country_region_time_intervention_sorted: 
    tp_dict = {}
    tp = country_region_time_intervention_sorted[k]  #all regions
    for k1 in tp:    #for every regions
        tp1 = tp[k1] #interventions
        #unit of one day
        
        temp_day = []
        for i in range(360):
            temp_day.append('')
            
        for k2 in tp1:    
            tp_0 = k2[0]  
            tp_1 = k2[1].split(' ')  
            tp0_split = tp_0.split('/')
            if int(tp0_split[1])<=30:
                tp_month_day  = (int(tp0_split[0])-1)*30 + int(tp0_split[1])
            else:
                tp_month_day  = (int(tp0_split[0])-1)*30 + 30
                
            if 'Re-opening' in tp_1:
                temp_day[tp_month_day] = '1'   #Re-opening
            else:
                temp_day[tp_month_day] = '0'   #Closure
        tp_dict[k1] = temp_day
    
    country_region_01[k] = tp_dict

##### 地区 Union 的方式

In [11]:
country_region_union_pre = {}
for item in country_region_01:
    case = country_region_01[item]
    mry_tp = {}
    for k in case:            
        region_tp = case[k]   
        
        if '0' in region_tp and '1' in region_tp:
      
            st = 0
            mry = []
            key = 0
            while True:
                tp_col = region_tp[st]
                #first closure
                if key==0:
                    if tp_col == '0':
                        mry.append((st,'0'))    #记录下第一个closure
                        st+=1
                        if st>=360:
                            break
                        key = 1                #用来判断找closure 还是 re-opening
                        continue
                    else:
                        st+=1
                        if st>=360:
                            break
                        continue
                else:
                    if tp_col == '1':
                        mry.append((st,'1'))    #记录下第一个re-opening
                        st+=1
                        key = 0
                        if st>=360:
                            break
                        continue
                    else:
                        st+=1
                        if st>=360:
                            break
                        continue
                        
            tp_l = len(mry)/2
            if tp_l>=1:
                tp_r = np.array(list(range(int(tp_l))))*2
                mry1 = []
                for i in tp_r:
                    m0 = mry[i][0]
                    m1 = mry[i+1][0]+1
                    for i1 in range(m0,m1):
                        mry1.append(i1)



                mry_tp[k] = (mry,mry1)
    if len(mry_tp)>0:
        country_region_union_pre[item] = mry_tp

In [28]:
print('regins without re-oepning Info',[k for k in set_countries if k not in country_region_union_pre])
tp = np.array([k for k in set_countries if k not in country_region_union_pre])
tp

regins without re-oepning Info ['Iceland', 'Bosnia and Herzegovina', 'Malaysia', 'Japan', 'Egypt', 'United States of America', 'Hungary', 'Belgium', 'Serbia', 'Kuwait', 'Albania', 'Canada', 'Syria', 'South Korea', 'Thailand', 'New Zealand', 'Kosovo', 'Czech Republic', 'Croatia', 'Indonesia', 'North Macedonia', 'Slovenia', 'Republic of Ireland', 'Sweden', 'Honduras', 'Romania', 'Taiwan', 'Mexico', 'Lithuania', 'Brazil', 'El Salvador', 'Kazakhstan']


array(['Iceland', 'Bosnia and Herzegovina', 'Malaysia', 'Japan', 'Egypt',
       'United States of America', 'Hungary', 'Belgium', 'Serbia',
       'Kuwait', 'Albania', 'Canada', 'Syria', 'South Korea', 'Thailand',
       'New Zealand', 'Kosovo', 'Czech Republic', 'Croatia', 'Indonesia',
       'North Macedonia', 'Slovenia', 'Republic of Ireland', 'Sweden',
       'Honduras', 'Romania', 'Taiwan', 'Mexico', 'Lithuania', 'Brazil',
       'El Salvador', 'Kazakhstan'], dtype='<U24')

In [13]:
info_without_reopening = {}
re_in_region = []
for k in tp:
    temp0 = country_region_01[k]
    rt = []
    for k0 in temp0:
        temp1 = temp0[k0]
        if '0' in temp1:
            tp1 = temp1.index('0')
            rt.append(tp1)
        else:
            re_in_region.append(k)
    if len(rt)>0:
        info_without_reopening[k] = min(rt)               

In [29]:
# Union 
country_region_union = {}
for k in country_region_union_pre:
    tp0 = country_region_union_pre[k]
    content = []
    for k0 in tp0:
        ct = tp0[k0][1]
        content = list(set(content).union(set(ct)))
    country_region_union[k] = content

In [15]:
for k in info_without_reopening:
    thrd = info_without_reopening[k]
    tp0 = []
    for i in range(thrd,360):
        tp0.append(i)
    country_region_union[k] = tp0

##### output of closure

In [16]:
country_closure = []
for k in country_region_union:
    country_closure.append((k,len(country_region_union[k])))
tp_pd = pd.DataFrame(country_closure)
# tp_pd.to_csv('country_closure_OR_OPERATION1124.csv',index=0,header=['country','duration'])

In [30]:
#second normalization
Normalized_country_region_union = {}
for k in country_region_union:
    tp = country_region_union[k]
    temp = []
    for i in range(360):
        temp.append(1)
        
    for k2 in tp:
        temp[k2]=0
        
    Normalized_country_region_union[k] = temp

In [18]:
#manually check sweden
temp = []
for i in range(98):
    temp.append(1)
for i in range(98,195):
    temp.append(0)
for i in range(195,360):
    temp.append(1)
    
Normalized_country_region_union['Sweden'] = temp  

In [19]:
Normalized_Data_closure = []
for k in Normalized_country_region_union:
    temp = [k]
    tp = Normalized_country_region_union[k]
    temp.extend(tp)
    Normalized_Data_closure.append(temp)

In [20]:
#Alphanumeric order
Normalized_Data_closure_alp = np.array(sorted(Normalized_Data_closure,key = lambda item:item[0][0],reverse=1))

In [21]:
data_closure_alp = Normalized_Data_closure_alp[:,1:]
Y_closure_alp = [k[0] for k in Normalized_Data_closure_alp]

In [22]:
Y_closure_alp[1] = 'USA'
Y_closure_alp[-5] = 'Bosnia'

In [23]:
len(Y_closure_alp)

56

In [24]:
y_ticks = []
yy = 0.5
for k in range(len(Y_closure_alp)):
    y_ticks.append(yy)
    yy+=1

In [25]:
Data_closure_alp = np.zeros(shape=(len(Y_closure_alp),360))
for i in range(len(Y_closure_alp)):
    for j in range(360):
        Data_closure_alp[i,j] = int(data_closure_alp[i,j])

In [26]:
col = []
for i in range(12):
    col.append((i+1)*30-15)

X_ticks = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    
Data1 = []
for i in range(len(Y_closure_alp)):
    tp = []
    for j in range(12):
        tp.append(0)
    Data1.append(tp)
Data1 = np.array(Data1)
# Data1

In [34]:
font1 = {'family': 'DejaVu Sans',
         'weight': 'bold',
         'style':'normal',
         'size': 15,
         }
Data = Data_closure_alp
Y_ticks = Y_closure_alp
fig, ax0 = plt.subplots(figsize=[14,20])
my_cmap = matplotlib.colors.ListedColormap(['orangered','skyblue'])
c = ax0.pcolor(Data,cmap=my_cmap,alpha = 0.6,edgecolors=None)
c = plt.xticks(ticks=col,labels=X_ticks)
c = plt.yticks(ticks=y_ticks,labels=Y_ticks,rotation=15)
ax0.xaxis.set_ticks_position('bottom')
for k in range(len(Y_closure_alp)):
    ax0.get_yticklabels()[k].set_fontweight("bold")
for k in range(12):
    ax0.get_xticklabels()[k].set_fontweight("bold")

ax1 = ax0.twiny() # the reason why it works
# ax2.set_ylabel('survival rate',font1)
ax1.pcolor(Data1,cmap=my_cmap,alpha = 0.1,edgecolors='k')
ax1.set_xticks([])
ax1.spines['top'].set_visible(False)

plt.bar([2],height=0,label='Closure Kingdergarten/Primary And Secondary School etc.',color='orangered',alpha=0.6)
plt.bar([2],height=0,label='Re-opening Kingdergarten/Primary And Secondary School etc.',color='skyblue',alpha=0.6)
legend_properties = {'weight':'bold','size':9.7}
plt.legend(loc=(-0.13,1.512), bbox_to_anchor=(0.1, 0.1, 6/10, 3/5), borderaxespad=0,prop = legend_properties, edgecolor='white',ncol=2)

##### main figure 3

In [40]:
import country_converter as coco
import latex

In [41]:
country_duration = pd.read_csv('country_closure_OR_OPERATION1124.csv')

In [42]:
NewAuthors = pd.read_csv('country/PeriodCountryNewAuthorsPreprints.csv')
SysProd = pd.read_csv('country/PeriodCountrySysProdPreprints.csv')
IndProd = pd.read_csv('country/PeriodCountryIndProdPreprints.csv')
pub2020 = pd.read_csv('country/PaperAuthorDateGenderSciAgeAll_2020.csv')

In [43]:
pub = pub2020[['PaperId','CountryCode']].drop_duplicates()

In [44]:
pub_CT = dict(Counter(pub.CountryCode))

In [45]:
target_countries = set([NewAuthors.Country[k] for k in range(len(NewAuthors.New_Authors)) if NewAuthors.New_Authors[k]>150])

In [46]:
target_countries = list(target_countries)
len(target_countries)

36

In [48]:
country_duration['code'] = coco.convert(names = list(country_duration.country.unique()), to='ISO2', not_found=None)

In [49]:
country_duration = np.array(country_duration)

In [32]:
countries_duration = {}
for k in country_duration:
    countries_duration[k[2]] = k[1]
countries_duration['SE'] = 97

In [51]:
x = [countries_duration[k] for k in target_countries if k in countries_duration]  #duration
#new authors
#2019-2020
newau2019 = np.array([list(NewAuthors.New_Authors[(NewAuthors.Country==k)&
             (NewAuthors.Date=='2019-01:2019-12')])[0]
             for k in target_countries if k in countries_duration])
# newau2019 = np.array([list(newau2019[i])[0] for i in range(len(newau2019))])
newau2020 = np.array([list(NewAuthors.New_Authors[(NewAuthors.Country==k)&
             (NewAuthors.Date=='2020-01:2020-12')])[0]
             for k in target_countries if k in countries_duration])
y_newau_20192020 = newau2020/newau2019
#2019-2020 (6 months)
newau2019_6 = np.array([list(NewAuthors.New_Authors[(NewAuthors.Country==k)&
             (NewAuthors.Date=='2019-01:2019-06')])[0]
             for k in target_countries if k in countries_duration])
# newau2019 = np.array([list(newau2019[i])[0] for i in range(len(newau2019))])
newau2020_6 = np.array([list(NewAuthors.New_Authors[(NewAuthors.Country==k)&
             (NewAuthors.Date=='2020-01:2020-06')])[0]
             for k in target_countries if k in countries_duration])
y_newau_20192020_6 = newau2020_6/newau2019_6

#2020-2021
newau2020_6 = np.array([list(NewAuthors.New_Authors[(NewAuthors.Country==k)&
             (NewAuthors.Date=='2020-01:2020-06')])[0]
             for k in target_countries if k in countries_duration])
# newau2019 = np.array([list(newau2019[i])[0] for i in range(len(newau2019))])
newau2021_6 = np.array([list(NewAuthors.New_Authors[(NewAuthors.Country==k)&
             (NewAuthors.Date=='2021-01:2021-06')])[0]
             for k in target_countries if k in countries_duration])
y_newau_20202021_6 = newau2021_6/newau2020_6
#2019-2021 (6 months)
y_newau_20192021_6 = newau2021_6/newau2019_6



#system productivity
#2019-2020
sys2019 = np.array([list(SysProd.Productivity[(SysProd.Country==k)&
             (SysProd.Date=='2019-01:2019-12')])[0]
             for k in target_countries if k in countries_duration])

sys2020 = np.array([list(SysProd.Productivity[(SysProd.Country==k)&
             (SysProd.Date=='2020-01:2020-12')])[0]
             for k in target_countries if k in countries_duration])

y_sysprod_20192020 = sys2020/sys2019
#2019-2020 (6 months)
sys2019_6 = np.array([list(SysProd.Productivity[(SysProd.Country==k)&
             (SysProd.Date=='2019-01:2019-06')])[0]
             for k in target_countries if k in countries_duration])

sys2020_6 = np.array([list(SysProd.Productivity[(SysProd.Country==k)&
             (SysProd.Date=='2020-01:2020-06')])[0]
             for k in target_countries if k in countries_duration])

y_sysprod_20192020_6 = sys2020_6/sys2019_6

#2020-2021
sys2020_6 = np.array([list(SysProd.Productivity[(SysProd.Country==k)&
             (SysProd.Date=='2020-01:2020-06')])[0]
             for k in target_countries if k in countries_duration])

sys2021_6 = np.array([list(SysProd.Productivity[(SysProd.Country==k)&
             (SysProd.Date=='2021-01:2021-06')])[0]
             for k in target_countries if k in countries_duration])

y_sysprod_20202021_6 = sys2021_6/sys2020_6
#2019-2021 (6 months)
y_sysprod_20192021_6 = sys2021_6/sys2019_6




#individual productivity
#2019-2020
ind2019 = np.array([list(IndProd.Productivity[(IndProd.Country==k)&
             (IndProd.Date=='2019-01:2019-12')])[0]
             for k in target_countries if k in countries_duration])

ind2020 = np.array([list(IndProd.Productivity[(IndProd.Country==k)&
             (IndProd.Date=='2020-01:2020-12')])[0]
             for k in target_countries if k in countries_duration])

y_indprod_20192020 = ind2020/ind2019
#2019-2020 (6 months)
ind2019_6 = np.array([list(IndProd.Productivity[(IndProd.Country==k)&
             (IndProd.Date=='2019-01:2019-06')])[0]
             for k in target_countries if k in countries_duration])

ind2020_6 = np.array([list(IndProd.Productivity[(IndProd.Country==k)&
             (IndProd.Date=='2020-01:2020-06')])[0]
             for k in target_countries if k in countries_duration])

y_indprod_20192020_6 = ind2020_6/ind2019_6

#2020-2021
ind2020_6 = np.array([list(IndProd.Productivity[(IndProd.Country==k)&
             (IndProd.Date=='2020-01:2020-06')])[0]
             for k in target_countries if k in countries_duration])

ind2021_6 = np.array([list(IndProd.Productivity[(IndProd.Country==k)&
             (IndProd.Date=='2021-01:2021-06')])[0]
             for k in target_countries if k in countries_duration])

y_indprod_20202021_6 = ind2021_6/ind2020_6
#2019-2021 (6 months)
y_indprod_20192021_6 = ind2021_6/ind2019_6

z = newau2020
country_name0 = [k for k in target_countries if k in countries_duration]
# print(country_name)
country_name = coco.convert(names = country_name0, to='ISO3', not_found=None) #'name_official'
country_group = coco.convert(names = country_name, to='name_official', not_found=None) #'name_official'

In [52]:
color_dict = {'United States of America':'mediumslateblue','Japan':'coral','Republic of Poland':'turquoise',
             'Kingdom of Belgium':'turquoise','French Republic':'turquoise','United Kingdom of Great Britain and Northern Ireland':'turquoise',
             "People's Republic of China":'coral','Republic of Austria':'turquoise','Canada':'mediumslateblue',
             'Republic of India':'coral','Kingdom of Spain':'turquoise','Federative Republic of Brazil':'mediumslateblue',
             'Republic of Finland':'turquoise','Kingdom of Denmark':'turquoise','Republic of Korea':'coral',
             'Italian Republic':'turquoise','Czech Republic':'turquoise','Kingdom of Sweden':'turquoise',
             'Kingdom of Norway':'turquoise','Swiss Confederation':'turquoise','Hellenic Republic':'turquoise',
             'Republic of Singapore':'coral','Portuguese Republic':'turquoise','United Mexican States':'mediumslateblue',
             'Federal Republic of Germany':'turquoise','Republic of China':'coral','Kingdom of the Netherlands':'turquoise'}

In [53]:
color_list = [color_dict[k] for k in country_group]

##### new figure： gender gap

In [54]:
#0222 data
NewData = pd.read_csv('new data 02222022/ind_productivity_country-gender-seniority.csv')

In [55]:
NewData.head(3)

,Country,Date,All,Female,Male,0-1,1-4,4-10,10-999
0,UZ,2015-07,0.285714,0.0,0.300000,0.0,0.0,2.0,0.444444
1,UZ,2015-08,0.421053,0.0,0.444444,0.0,0.2,2.0,0.555556
2,UZ,2015-09,0.444444,0.0,0.470588,0.0,0.2,2.0,0.625000


In [56]:
x = [countries_duration[k] for k in target_countries if k in countries_duration]  #duration
z = newau2020
country_name0 = [k for k in target_countries if k in countries_duration]
# print(country_name)
country_name = coco.convert(names = country_name0, to='ISO3', not_found=None) #'name_official'
country_group = coco.convert(names = country_name, to='name_official', not_found=None) #'name_official'

ind2021_6_Male = np.array([list(NewData.Male[(NewData.Country==k)&(NewData.Date=='2021-06')])[0]
             for k in target_countries if k in countries_duration])
ind2021_6_Female = np.array([list(NewData.Female[(NewData.Country==k)&(NewData.Date=='2021-06')])[0]
             for k in target_countries if k in countries_duration])

ind2021_3_Male = np.array([list(NewData.Male[(NewData.Country==k)&(NewData.Date=='2021-03')])[0]
             for k in target_countries if k in countries_duration])
ind2021_3_Female = np.array([list(NewData.Female[(NewData.Country==k)&(NewData.Date=='2021-03')])[0]
             for k in target_countries if k in countries_duration])




ind2020_3_Male = np.array([list(NewData.Male[(NewData.Country==k)&(NewData.Date=='2020-03')])[0]
             for k in target_countries if k in countries_duration])
ind2020_3_Female = np.array([list(NewData.Female[(NewData.Country==k)&(NewData.Date=='2020-03')])[0]
             for k in target_countries if k in countries_duration])

ind2020_2_Male = np.array([list(NewData.Male[(NewData.Country==k)&(NewData.Date=='2020-02')])[0]
             for k in target_countries if k in countries_duration])
ind2020_2_Female = np.array([list(NewData.Female[(NewData.Country==k)&(NewData.Date=='2020-02')])[0]
             for k in target_countries if k in countries_duration])


ind2019_3_Male = np.array([list(NewData.Male[(NewData.Country==k)&(NewData.Date=='2019-03')])[0]
             for k in target_countries if k in countries_duration])
ind2019_3_Female = np.array([list(NewData.Female[(NewData.Country==k)&(NewData.Date=='2019-03')])[0]
             for k in target_countries if k in countries_duration])

ind2019_6_Male = np.array([list(NewData.Male[(NewData.Country==k)&(NewData.Date=='2019-06')])[0]
             for k in target_countries if k in countries_duration])
ind2019_6_Female = np.array([list(NewData.Female[(NewData.Country==k)&(NewData.Date=='2019-06')])[0]
             for k in target_countries if k in countries_duration])

In [109]:
x = x
z0 = [pub_CT[k] for k in country_name0]
y1 = y_indprod_20192021_6
y2 = (ind2021_6_Male - ind2021_6_Female)/(ind2020_3_Male - ind2020_3_Female)
# y2 = y_indprod_20192020_6
Y = [y1,y2]
# Y_Label = [
#     'Individual productivity Jan-June 2020\nIndividual productivity Jan-June 2019',
#     'Individual productivity Jan-June 2021\nIndividual productivity Jan-June 2019'
#            ]
Y_Label = [
    'Individual productivity\n Jan-Jun 2021 / Jan-Jun 2019',
    'Gendered productivity gap\n Jun 2021 / Mar 2020'
           ]
z = [k/500 for k in z0]
Title = ['Individual productivity','Individual productivity']

In [110]:
import matplotlib.gridspec as gridspec

In [33]:
l_index = ['b)','c)']
sns.set(style='ticks', font_scale=0.75, font='Arial')
fig = plt.figure(tight_layout=True,figsize=[10,7])  #22,14
gs = gridspec.GridSpec(2, 2)

ax0 = fig.add_subplot(gs[:, 0])
Data = Data_closure_alp
Y_ticks = Y_closure_alp
my_cmap = matplotlib.colors.ListedColormap(['lightcoral','mediumspringgreen']) #'orangered','skyblue','aquamarine''''
c = ax0.pcolor(Data,cmap=my_cmap,alpha = 0.6,edgecolors=None)
c = plt.xticks(ticks=col,labels=X_ticks)
c = plt.yticks(ticks=y_ticks,labels=Y_ticks,rotation=0)
ax0.xaxis.set_ticks_position('bottom')
# for k in range(len(Y_closure_alp)):
#     ax0.get_yticklabels()[k].set_fontweight("bold")
# for k in range(12):
#     ax0.get_xticklabels()[k].set_fontweight("bold")

ax1 = ax0.twiny() # the reason why it works
# ax2.set_ylabel('survival rate',font1)
ax1.pcolor(Data1,cmap=my_cmap,alpha = 0.1,edgecolors='k')
ax1.set_xticks([])
ax1.spines['top'].set_visible(False)

ax0.bar([2],height=0,label='daycares and primary schools closed',color='lightcoral',alpha=0.6)
ax0.bar([2],height=0,label='daycares and primary schools open',color='mediumspringgreen',alpha=0.6)
# legend_properties = {'weight':'normal','size':8}
ax0.legend(loc=(-0.21,-0.3), bbox_to_anchor=(0.1, 0.1, 6/10, 3/5), borderaxespad=0,edgecolor='white',ncol=2,fontsize=6.5)#prop = legend_properties, 
font = { 'weight': 'bold',
             'size': 15,
        'family':'Arial'
             }
ax0.text(-0.27, 1.0,'a)', fontdict=font,ha='center', va='center', transform=ax0.transAxes)
# font = { 'weight': 'normal',
#          'size': 15,
#          }



row_list = [0,1]
for i in row_list:
    ax = fig.add_subplot(gs[i, 1])
    y = Y[i]
    scatter = ax.scatter(x, y, s=z, alpha=0.8,c=color_list,edgecolor='white')
    ax.plot([0,360],[1,1],ls='--',color='grey')
#     handles, labels = scatter.legend_elements(prop="sizes", alpha=0.6)
#     legend2 = ax.legend(handles, labels, loc="upper right", title="Sizes")
    manually = [21,9,17,10,24,19,4,18,23,11,16,12,2,7,8,20,25,26,6,3,13]     #若图片作大改动，这里需要改变
    for ii in range(len(x)):
        if ii not in manually:
            ax.text(x[ii]*0.99, y[ii]*1.005, country_name[ii], 
                    fontsize=5, color = "dimgrey", style = "italic", weight = "light",
                    verticalalignment='center', horizontalalignment='right',rotation=0) #给散点加标签
#     fts = 7
#     #挨个调整国家的图例位置
#     #USA
#     ii = 0
#     ax.text(x[ii]*1.05, y[ii]*0.975, country_name[ii], fontsize=fts, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) #给散点加标签
    ii = 2
    ax.text(x[ii]*1.08, y[ii]*1.0, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 3
    ax.text(x[ii]*1.15, y[ii]*0.99, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 4
    ax.text(x[ii]*0.99, y[ii]*0.99, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
#     ii = 5
#     ax.text(x[ii]*1.17, y[ii]*0.99, country_name[ii], fontsize=10, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 6 #JPN
    ax.text(x[ii]*1.05, y[ii]*1.00, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 7 #TWN
    ax.text(x[ii]*1.05, y[ii]*1.01, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 8
    ax.text(x[ii]*0.99, y[ii]*0.97, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 9
    ax.text(x[ii]*1.00, y[ii]*0.96, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 10
    ax.text(x[ii]*1.2, y[ii]*0.985, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 11  #ITA
    ax.text(x[ii]*1.08, y[ii]*1.00, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 12
    ax.text(x[ii]*1.02, y[ii]*0.985, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 13
    ax.text(x[ii]*0.975, y[ii]*1.015, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
#     ii = 14
#     ax.text(x[ii]*0.99, y[ii]*1.005, country_name[ii], fontsize=10, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
#     ii = 15
#     ax.text(x[ii]*0.99, y[ii]*1.005, country_name[ii], fontsize=10, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 16 #INA
    ax.text(x[ii]*1.06, y[ii]*1.009, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 17 #CAN
    ax.text(x[ii]*0.99, y[ii]*0.97, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 18
    ax.text(x[ii]*1.19, y[ii]*0.995, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 19
    ax.text(x[ii]*1.0, y[ii]*1.005, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 20 #JAN
    ax.text(x[ii]*1.05, y[ii]*1.005, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 21 #ITA
    ax.text(x[ii]*1.08, y[ii]*1.0, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
#     ii = 22
#     ax.text(x[ii]*0.99, y[ii]*1.005, country_name[ii], fontsize=10, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 23 #USA
    ax.text(x[ii]*1.0, y[ii]*1.00, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 24
    ax.text(x[ii]*1.1, y[ii]*1.05, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 25 #BRA
    ax.text(x[ii]*1.05, y[ii]*1.00, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
    ii = 26  #CAN
    ax.text(x[ii]*1.05, y[ii]*1.01, country_name[ii], fontsize=5, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
# #     ii = 27
# #     ax.text(x[ii]*0.99, y[ii]*1.005, country_name[ii], fontsize=10, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
# #     ax.text(x[ii]*0.99, y[ii]*1.005, country_name[ii], fontsize=10, color = "dimgrey", style = "italic", weight = "light",verticalalignment='center', horizontalalignment='right',rotation=0) 
        
    
        
#     ax.set_title('new authors',font) 
    ax.set_xlabel('days of closure')
    ax.set_ylabel(Y_Label[i])
    if i==0:
        ax.set_ylim([0.6,1.4])
    else:
        ax.set_ylim([-6,6])
#     ax.set_title(Title[i], font)
    ax.grid(True,color='lightgrey',ls='dotted')
    
    ax.text(-0.14, 1.0,l_index[i], fontdict=font,ha='center', va='center', transform=ax.transAxes)
    g = ax.scatter([],[], s=1**0.5, marker='o', color='white')
    gll = ax.scatter([],[], s=20000/500, marker='o', edgecolor=None,color='darkgrey')
    gl = ax.scatter([],[], s=100000/500, marker='o', edgecolor=None,color='darkgrey')
    ga = ax.scatter([],[], s=400000/500, marker='o', edgecolor=None,color='darkgrey')
    cn = ax.scatter([],[], s=1**0.5, marker='o', color='white')
    cas = ax.scatter([],[], s=50000/500, marker='o', edgecolor=None,color='coral')
    ceu = ax.scatter([],[], s=50000/500, marker='o', edgecolor=None,color='turquoise')
    cam = ax.scatter([],[], s=50000/500, marker='o', edgecolor=None,color='mediumslateblue')
    #mediumslateblue coral turquoise
    if i==0:
        legend = ax.legend((ga,gl,gll,cas,ceu,cam),
               ('  500000', '  100000', '  20000',
               'Asia','Europe','America'),
               scatterpoints=1,
#                loc='upper right',
               loc=(0.45,0.5),
               ncol=2,
               fontsize=7.5,
                handlelength=2, borderpad=1,labelspacing=2.2)#handlelength=1.5, borderpad=0.5, labelspacing=2
        frame = legend.get_frame() 
        frame.set_alpha(1) 
        frame.set_facecolor('none') 
        legend.get_frame().set_linewidth(0.0)

        ax.text(0.61, 0.965,'Publications 2020',ha='center', va='center', transform=ax.transAxes,fontsize=7.5) #fontdict=font,
        ax.text(0.84, 0.965,'Continent',ha='center', va='center', transform=ax.transAxes,fontsize=7.5) #fo
# fig.savefig('NPI_panel.pdf')
